# Question

```python


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
# Figure out how to import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np 


# In[ ]:


# !wget --no-check-certificate \
#     https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
#     -O /tmp/sonnets.txt


# In[ ]:


tokenizer = Tokenizer()

data = open('/tmp/sonnets/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)


# In[ ]:


model = Sequential()
model.add(# Your Embedding Layer)
model.add(# An LSTM Layer)
model.add(# A dropout layer)
model.add(# Another LSTM Layer)
model.add(# A Dense Layer including regularizers)
model.add(# A Dense Layer)
# Pick an optimizer
model.compile(# Pick a loss function and an optimizer)
print(model.summary())


# In[ ]:


history = model.fit(predictors, label, epochs=100, verbose=1)


# In[ ]:


import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()


# In[ ]:


seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)


```

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.utils as ku 
import numpy as np 

In [14]:
from tensorflow.keras.regularizers import l2

In [4]:
data = open('/tmp/sonnets/sonnets.txt').read()
corpus = data.lower().split("\n")

In [5]:
corpus[100]

'why lovest thou that which thou receivest not gladly,'

In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [8]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [9]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [13]:
total_words

3211

In [27]:
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=l2()))
model.add(BatchNormalization())
model.add(Dense(total_words, activation='softmax'))
# Pick an optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 11, 64)            205504    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 11, 256)           197632    
_________________________________________________________________
dropout_6 (Dropout)          (None, 11, 256)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                82176     
_________________________________________________________________
dense_11 (Dense)             (None, 1605)              104325    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1605)              6420      
_________________________________________________________________
dense_12 (Dense)             (None, 3211)             

In [ ]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 5s 10ms/step - loss: 6.9282 - accuracy: 0.0279
Epoch 2/100
484/484 [==============================] - 5s 10ms/step - loss: 6.3667 - accuracy: 0.0398
Epoch 3/100
484/484 [==============================] - 5s 10ms/step - loss: 6.1035 - accuracy: 0.0533
Epoch 4/100
484/484 [==============================] - 5s 10ms/step - loss: 5.8612 - accuracy: 0.0644
Epoch 5/100
484/484 [==============================] - 5s 10ms/step - loss: 5.6173 - accuracy: 0.0799
Epoch 6/100
484/484 [==============================] - 5s 10ms/step - loss: 5.3775 - accuracy: 0.0898
Epoch 7/100
484/484 [==============================] - 5s 10ms/step - loss: 5.1514 - accuracy: 0.1032
Epoch 8/100
484/484 [==============================] - 5s 10ms/step - loss: 4.9343 - accuracy: 0.1224
Epoch 9/100
484/484 [==============================] - 5s 9ms/step - loss: 4.7299 - accuracy: 0.1375
Epoch 10/100
484/484 [==============================] - 5s 10ms/step - loss: 4.5283

484/484 [==============================] - 5s 10ms/step - loss: 0.9729 - accuracy: 0.7948
Epoch 77/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9795 - accuracy: 0.7934
Epoch 78/100
484/484 [==============================] - 5s 9ms/step - loss: 0.9759 - accuracy: 0.7936
Epoch 79/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9662 - accuracy: 0.7954
Epoch 80/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9620 - accuracy: 0.7962
Epoch 81/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9453 - accuracy: 0.8033
Epoch 82/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9634 - accuracy: 0.7955
Epoch 83/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9556 - accuracy: 0.7956
Epoch 84/100
484/484 [==============================] - 5s 9ms/step - loss: 0.9315 - accuracy: 0.8042
Epoch 85/100
484/484 [==============================] - 5s 10ms/step - loss: 0.9401 - ac